In [1]:
import pandas as pd
import numpy as np
import math
import re
from sqlalchemy import create_engine
from xml.etree import ElementTree as ET
import pickle
import pyodbc

In [2]:
# データベースの情報
server = 'gciteam16.database.windows.net'
database = 'mynavi-database'
username = 'gciteam16'
password = 'Password0'
port =1433

In [3]:
# 接続エンジンの作成
engine_config = "?driver=ODBC+Driver+13+for+SQL+Server?charset=shift-jis"
db_settings = {
    "host": server,
    "database": database,
    "user": username,
    "password": password,
    "port":port,
    "config_query":engine_config
}
engine = create_engine('mssql+pyodbc://{user}:{password}@{host}:{port}/{database}{config_query}'.format(**db_settings))

In [182]:
#data = pd.read_csv("../../gci_project_storage/mynavi_data.csv", encoding="SHIFT-JIS")
query = "SELECT * FROM raw_data_table"
data = pd.read_sql_query(query,engine)

query = "SELECT * FROM address_master"
address_master = pd.read_sql_query(query,engine)

query = "SELECT * FROM station_master"
#address_master = pd.read_sql_query(query,engine)


In [183]:
data.shape

(46469, 110)

# 目次：
<br>
・緯度経度の追加<br>
・築年数（築年）の取得
・近所の施設⽬での距離を取得<br>
・間取り情報の抽出<br>
・⽋損値にフラグを⽴てて、使えるデータに変換<br>
・CSVファイル(mynavi_data2.csv)に落とす<br>

<h3>緯度経度の追加

In [184]:
get_lat_from_address = lambda address:list(address_master.latitude [address_master.address==address])[0]
get_lng_from_address = lambda address:list(address_master.longitude[address_master.address==address])[0]

In [185]:
data["address_lat"] = data.address.apply(get_lat_from_address)
data["address_lng"] = data.address.apply(get_lng_from_address)

In [186]:
data = data[data.address_lat.apply(math.isnan)==False]

with open('../../gci_project_storage/result_list_missed.pickle', mode='wb') as f:
    pickle.dump(data.address[data_lat.apply(math.isnan)].unique(), f)

print(data_address.iloc[0:1000,2].apply(math.isnan).sum())
print(data_address.iloc[1000:2000,2].apply(math.isnan).sum())
print(data_address.iloc[2000:4000,2].apply(math.isnan).sum())
print(data_address.iloc[5000:6400,2].apply(math.isnan).sum())
print(data_address.iloc[6400:,2].apply(math.isnan).sum())

data_address = pd.read_csv('../../gci_project_storage/data_address.csv', encoding="SHIFT-JIS")[["address","longitude","latitude"]]

<h3>築年数（築年）の取得

In [187]:
get_year_build = lambda date:int(re.findall("([0-9]+)年[0-9]+月",date)[0]) if len( re.findall("([0-9]+)年[0-9]+月",date) )>0 else None

In [188]:
data["year_built"] = data.date.apply(get_year_build)
sum( data.year_built.apply(math.isnan) )

9

In [189]:
# 築年のないログ（9個）を削除
data = data[data.year_built.apply(math.isnan)==False]

<h3>近所の施設⽬での距離を取得

In [190]:
#近所の施設の一覧
get_facilities = lambda x:re.findall("【(\S+?)】",x)
all_facilities = []
for facility in data.surroundings.dropna().apply(get_facilities):
    all_facilities += facility
pd.Series(all_facilities).unique()

array(['小学校', '学校', 'コンビニ', 'スーパー', 'ドラッグストア', '飲食店', '郵便局', '公園', '病院',
       'レンタルビデオ', 'その他', '幼稚園・保育園', '図書館', '大学', '銀行', '総合病院', 'クリーニング',
       'デパート', 'コインパーキング', '駒込駅', '11', '6／19オープン', '1／30オープン',
       '両国内科皮膚科整形外科', '5／17オープン', '4／7オープン', '7／23オープン', '9／30オープン',
       '5／20オープン', '5／19オープン', '8／18オープン', '6／17オープン', '8／23オープン', '月極駐車場',
       '4／22オープン', '3／31オープン', '3／29オープン', '6／9オープン', '5／15オープン',
       '3／25オープン'], dtype=object)

In [191]:
facilities = ['幼稚園・保育園', '小学校', '学校', 'コンビニ', 'スーパー', '郵便局', '図書館',
       'ドラッグストア', '飲食店', '銀行', '総合病院', '病院', '公園', '大学', 'レンタルビデオ',
       'クリーニング', 'デパート']
new_colnames = ['kindergarten', 'elementary_school', 'school', 'convenient_store', 'super_market', 'post_office', 'library',
       'drug_store', 'restaurant', 'bank', 'general_hospital', 'hospital', 'park', 'university', 'rental_video',
       'cleaning_shop', 'department_store']
data.surroundings[0]

'【小学校】船橋小学校(348m)【学校】船橋中学校(410m)【コンビニ】コンビニ(441m)【スーパー】スーパー(490m)'

In [192]:
#施設までの距離を取得する関数
get_distance = lambda facility, text: int(re.findall("【"+facility+"】\S+?\(([0-9]+)m",text)[0]) if len(re.findall("【"+facility+"】\S+?\(([0-9]+)m",text))==1 else None
get_distance(facilities[1], data.surroundings[0])

348

In [193]:
#施設までの距離を持った新しいデータフレーム作成
df_new = pd.DataFrame(index=data.index, columns=new_colnames)

#数字を実際に取得して埋める
for facility,colname in zip(facilities, new_colnames):
    get_distance_fac = lambda text: get_distance(facility, text)
    df_new[colname][data.surroundings.isnull()==False] = data[data.surroundings.isnull()==False].surroundings.apply(get_distance_fac)

In [194]:
df_new.head()

,kindergarten,elementary_school,school,convenient_store,super_market,post_office,library,drug_store,restaurant,bank,general_hospital,hospital,park,university,rental_video,cleaning_shop,department_store
0,NaN,348,410,441,490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,152,568,244,322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,152,568,244,322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,562,795,415,NaN,676,NaN,NaN,NaN,846,527,NaN,853,NaN,NaN
5,NaN,991,1012,NaN,325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [195]:
#横結合で、元のデータフレームを拡張
data = pd.concat([data, df_new], axis=1)
data.shape

(30728, 130)

<h3>間取り情報の抽出

In [196]:
data.layout.unique()

array(['1LDK', '2LDK', '1R', '3LDK', '1K', '1DK', '2K', '2DK', '3DK',
       '1LDK+S(納戸)', '2LDK+S(納戸)', '4LDK', '8K', '3DK+S(納戸)', '2DK+S(納戸)',
       '3K+S(納戸)', '5LDK', '1K+S(納戸)', '1DK+S(納戸)', '3LDK+S(納戸)',
       '4LDK+S(納戸)', '3K', '2R', '4DK', '6LDK', '1LK', '5LDK+S(納戸)',
       '5DK+S(納戸)', '5DK', '2K+S(納戸)', '5K', '4DK+S(納戸)', '7LDK'], dtype=object)

In [197]:
#情報を抽出する関数
has_K = lambda text: re.findall("K",text)!=None
has_L = lambda text: re.findall("L",text)!=None
has_D = lambda text: re.findall("D",text)!=None
has_S = lambda text: re.findall("S",text)!=None
get_Num_Rooms = lambda text: int(re.findall("[0-9]+",text)[0])

has_K('1K'),get_Num_Rooms('4LDK+S(納戸)')

(True, 4)

In [198]:
#追加したいカラム名
new_colnames = ["has_K","has_L","has_D","has_S","the_number_of_rooms"]

#間取りの情報を持った新しいデータフレーム作成
df_new = pd.DataFrame(index=data.index, columns=new_colnames)

#数字を実際に取得して埋める
df_new["has_K"][data.layout.isnull()==False] = data[data.layout.isnull()==False].layout.apply(has_K)
df_new["has_L"][data.layout.isnull()==False] = data[data.layout.isnull()==False].layout.apply(has_L)
df_new["has_D"][data.layout.isnull()==False] = data[data.layout.isnull()==False].layout.apply(has_D)
df_new["has_S"][data.layout.isnull()==False] = data[data.layout.isnull()==False].layout.apply(has_S)
df_new["the_number_of_rooms"][data.layout.isnull()==False] = data[data.layout.isnull()==False].layout.apply(get_Num_Rooms)


In [199]:
#横結合で、元のデータフレームを拡張
data = pd.concat([data, df_new], axis=1)
data.shape

(30728, 135)

In [200]:
data.columns

Index(['traffic', 'address', 'rent', 'area', 'deposit', 'key_money',
       'brokerage_fee', 'date', 'layout', 'layout_detail',
       ...
       'park', 'university', 'rental_video', 'cleaning_shop',
       'department_store', 'has_K', 'has_L', 'has_D', 'has_S',
       'the_number_of_rooms'],
      dtype='object', length=135)

<h3>⽋損値にフラグを⽴てて、使えるデータに変換

In [201]:
#欠損してる説明変数一覧：
null_colnames = pd.Series([
     "locality"         ,"buildings_height"    ,"buildings_undergrand" ,"total_units"
    ,"balcony"          ,"shower"              ,"bath_drier"           ,"washlet"
    ,"toilet_style"     ,"bathhouse"           ,"water_heater"         ,"system_kitchen"
    ,"counter_kitchen"  ,"independent_kitchen" ,"L_kitchen"            ,"refrigerator" 
    ,"gas_stove"        ,"IH_stove"            ,"electric_stove"       ,"stove_num"
    ,"office"           ,"instrument"          ,"togather"             ,"share"
    ,"foreigner"        ,"pet"                 ,"male"                 ,"female"
    , "kindergarten"    ,"elementary_school"   ,"school"               ,"convenient_store"
    ,"super_market"     ,"post_office"         ,"library"              ,"drug_store"
    ,"restaurant"       ,"bank"                ,"general_hospital"     ,"hospital"
    ,"park"             ,"university"          ,"rental_video"         ,"cleaning_shop"
    ,"department_store" ,"the_number_of_rooms" ,"year_built"
])
len(null_colnames)

47

In [202]:
#欠損地にフラグを立ててカラムを追加
data["isNull_" + null_colnames] = data[null_colnames].isnull()

In [203]:
data.shape

(30728, 182)

In [204]:
data[null_colnames + "_Null_to_Zero"] = data[null_colnames].fillna(0)

In [205]:
data.shape

(30728, 229)

<h3>最寄駅情報の抽出

In [206]:
# 駅名と徒歩何分かを取得する関数
remove_bracket = lambda text :re.sub(u'\(.+?\)','',re.sub(u'（.+?）', '',re.sub(u'「」', '',text))) 
remove_question_mark = lambda text :re.sub(u'\?','',text)
remove_noise = lambda text:remove_bracket(remove_question_mark(text))
remove_dot = lambda text :re.sub(u'・.+?線','',text)#"・中央線東京駅"みたいなのを除く

get_station_info_0 = lambda text: (
    re.findall(u"線(\S+?駅).*?徒歩([0-9]+?)分", remove_noise(text)) 
    + re.findall(u"ライン(\S+?駅).*?徒歩([0-9]+?)分", remove_noise(text)) 
    + re.findall(u"ライナー(\S+?駅).*?徒歩([0-9]+?)分", remove_noise(text))
)
get_station_info_1 = lambda text:[(re.sub(u"「",u"",st_tuple[0]),st_tuple[1]) for st_tuple in get_station_info_0(text)]
get_station_info = lambda text:{remove_dot(t[0]):int(t[1]) for t in get_station_info_1(text)} if len(get_station_info_1(text)) > 0 else {}
get_station_name = lambda text:[remove_dot(t[0]) for t in get_station_info_1(text)] if len(get_station_info_1(text)) > 0 else []
station_names = []

# 駅の総数を調べる
count_no_st = 0
for text in data.traffic:
    station_names += get_station_name(text)
    if(len(get_station_name(text))==0):
        count_no_st += 1
station_names = pd.Series(pd.Series(station_names).unique())

print("# Station : {}".format(len(station_names)))
print("# Rental Without Station : {}".format(count_no_st)) #＜-徒歩でなく、バスで移動の物件はNoneが返り値

# Station : 287
# Rental Without Station : 63


In [207]:
station_info = data.traffic.apply(get_station_info)

In [208]:
# 近くの駅までの所要時間の逆数のデータフレーム（説明変数に使う）
df_new = pd.DataFrame(index=data.index, columns=station_names).astype(np.float)

In [209]:
# 駅ごとにそれらを求める関数(徒歩0分の場合は、inverse:=2とした)
get_inverse_dist_to_st = lambda arg_dict, st_name : (1.0/arg_dict[st_name] if arg_dict[st_name]!=0 else 2) if st_name in arg_dict.keys() else 0

In [210]:
for st_name in station_names:
    this_st = st_name
    get_inverse_dist_to_this_st = lambda arg_dict: get_inverse_dist_to_st(arg_dict ,this_st)
    df_new[st_name] = station_info.apply(get_inverse_dist_to_this_st)

In [211]:
#横結合で、元のデータフレームを拡張
data = pd.concat([data, df_new], axis=1)
data.shape

(30728, 516)

<h3>DBとpickleファイル(mynavi_data2.pickle)に落とす

In [12]:
table_name = "whole_data_table"
#data.to_sql(table_name, con=engine, if_exists="replace")

In [ ]:
query = "SELECT * FROM whole_data_table"
#data = pd.read_sql(query, con=engine)

In [212]:
# data.to_csv("../../gci_project_storage/mynavi_data2.csv", sep=",", encoding="UTF-8")
# 上手くいかないのでpickle
import pickle
with open('../../gci_project_storage/data.pickle', mode='wb') as f:
     pickle.dump(data, f)

<h3> 分析用データの作成

In [5]:
analytical_columns = pd.Series([
    "url"         ,"rent"         ,"area"        ,"flooring"    ,"year_built"
    ,"parking"     ,"structure"   ,"air_conditioner","auto_lock"
    ,"bath_toilet" ,"reheating"   ,"wash_basin"  ,"washing_machine"
    ,"top_floor"   ,"corner_room" ,"immediate"   ,"upper"
    ,"direction_1" ,"direction_2" ,"direction_3" ,"direction_4"
    ,"direction_5" ,"direction_6" ,"direction_7" ,"direction_8"
    ,"structure_1" ,"structure_2" ,"structure_3" ,"structure_4"
    ,"structure_5" ,"structure_6" ,"structure_7" ,"structure_8"
    ,"structure_9" ,"structure_10","structure_11"
    ,"status_1"    ,"status_2"    ,"status_3"    ,"status_4"    
    ,"transaction_type_1","transaction_type_2","transaction_type_3"
    ,"transaction_type_4","transaction_type_5","transaction_type_6"
    ,"bath_style_0","bath_style_1","bath_style_2"
    ,"address_lat" ,"address_lng"
    ,"has_K"       ,"has_L"       ,"has_D"       ,"has_S"
    ,"the_number_of_rooms"
    
    ,"isNull_locality"            ,"isNull_buildings_height"
    ,"isNull_buildings_undergrand","isNull_total_units"
    ,"isNull_balcony"             ,"isNull_shower"
    ,"isNull_bath_drier"          ,"isNull_washlet"
    ,"isNull_toilet_style"        ,"isNull_bathhouse"
    ,"isNull_water_heater"        ,"isNull_system_kitchen"
    ,"isNull_counter_kitchen"     ,"isNull_independent_kitchen"
    ,"isNull_L_kitchen"           ,"isNull_refrigerator"
    ,"isNull_gas_stove"           ,"isNull_IH_stove"
    ,"isNull_electric_stove"     ,"isNull_stove_num"
    ,"isNull_office"              ,"isNull_instrument"
    ,"isNull_togather"            ,"isNull_share"
    ,"isNull_foreigner"           ,"isNull_pet"
    ,"isNull_male"                ,"isNull_female"
    ,"isNull_kindergarten"        ,"isNull_elementary_school"
    ,"isNull_school"              ,"isNull_convenient_store"
    ,"isNull_super_market"        ,"isNull_post_office"
    ,"isNull_library"             ,"isNull_drug_store"
    ,"isNull_restaurant"          ,"isNull_bank"
    ,"isNull_general_hospital"    ,"isNull_hospital"
    ,"isNull_park"                ,"isNull_university"
    ,"isNull_rental_video"        ,"isNull_cleaning_shop"
    ,"isNull_department_store"    ,"isNull_the_number_of_rooms"
   
    ,"locality_Null_to_Zero"      ,"buildings_height_Null_to_Zero"
    ,"buildings_undergrand_Null_to_Zero"    ,"total_units_Null_to_Zero"
    ,"balcony_Null_to_Zero"       ,"shower_Null_to_Zero"
    ,"bath_drier_Null_to_Zero"    ,"washlet_Null_to_Zero"
    ,"toilet_style_Null_to_Zero"  ,"bathhouse_Null_to_Zero"
    ,"water_heater_Null_to_Zero"  ,"system_kitchen_Null_to_Zero"
    ,"counter_kitchen_Null_to_Zero"    ,"independent_kitchen_Null_to_Zero"
    ,"L_kitchen_Null_to_Zero"     ,"refrigerator_Null_to_Zero"
    ,"gas_stove_Null_to_Zero"     ,"IH_stove_Null_to_Zero"
    ,"electric_stove_Null_to_Zero","stove_num_Null_to_Zero"
    ,"office_Null_to_Zero"        ,"instrument_Null_to_Zero"
    ,"togather_Null_to_Zero"      ,"share_Null_to_Zero"
    ,"foreigner_Null_to_Zero"     ,"pet_Null_to_Zero"
    ,"female_Null_to_Zero"        ,"kindergarten_Null_to_Zero"
    ,"elementary_school_Null_to_Zero"    ,"school_Null_to_Zero"
    ,"convenient_store_Null_to_Zero"    ,"super_market_Null_to_Zero"
    ,"post_office_Null_to_Zero"   ,"library_Null_to_Zero"
    ,"drug_store_Null_to_Zero"    ,"restaurant_Null_to_Zero"
    ,"bank_Null_to_Zero"          ,"general_hospital_Null_to_Zero"
    ,"hospital_Null_to_Zero"    ,"park_Null_to_Zero"
    ,"university_Null_to_Zero"    ,"rental_video_Null_to_Zero"
    ,"cleaning_shop_Null_to_Zero" ,"department_store_Null_to_Zero"
    ,"the_number_of_rooms_Null_to_Zero"

    ,"千歳船橋駅"     ,"八幡山駅"       ,"千歳烏山駅"     ,"芦花公園駅" 
    ,"桜新町駅"       ,"駒沢大学駅"     ,"世田谷駅"       ,"用賀駅"     
    ,"二子玉川駅"     ,"祖師ケ谷大蔵駅"  ,"桜上水駅"       ,"下高井戸駅" 
    ,"上北沢駅"       ,"松陰神社前駅"   ,"若林駅"         ,"久我山駅"   
    ,"吉祥寺駅"       ,"東松原駅"       ,"新代田駅"       ,"明大前駅"   
    ,"等々力駅"       ,"尾山台駅"       ,"自由が丘駅"     ,"豪徳寺駅"       
    ,"山下駅"         ,"宮の坂駅"       ,"成城学園前駅"    ,"三軒茶屋駅" 
    ,"西太子堂駅"      ,"笹塚駅"         ,"代田橋駅"       ,"下北沢駅"   
    ,"上町駅"         ,"経堂駅"         ,"松原駅"         ,"仙川駅"     
    ,"梅ケ丘駅"       ,"世田谷代田駅"    ,"池ノ上駅"       ,"上野毛駅"   
    ,"喜多見駅"       ,"学芸大学駅"     ,"祐天寺駅"       ,"池尻大橋駅" 
    ,"九品仏駅"       ,"春日駅"         ,"後楽園駅"       ,"本郷三丁目駅"
    ,"田端駅"         ,"千駄木駅"       ,"本駒込駅"       ,"護国寺駅"   
    ,"江戸川橋駅"     ,"茗荷谷駅"       ,"千石駅"         ,"巣鴨駅"     
    ,"神楽坂駅"       ,"飯田橋駅"       ,"駒込駅"         ,"新大塚駅"   
    ,"西日暮里駅"     ,"御茶ノ水駅"     ,"新御茶ノ水駅"    ,"白山駅"     
    ,"東大前駅"       ,"根津駅"         ,"水道橋駅"       ,"湯島駅"     
    ,"御徒町駅"       ,"上野御徒町駅"    ,"大塚駅"         ,"向原駅"     
    ,"東池袋駅"       ,"日暮里駅"       ,"早稲田駅"       ,"秋葉原駅"   
    ,"池袋駅"         ,"末広町駅"       ,"上野広小路駅"    ,"快速日暮里駅"    
    ,"牛込神楽坂駅"    ,"雑司が谷駅"     ,"鬼子母神前駅"    ,"目白駅"     
    ,"東池袋四丁目駅"  ,"上野駅"         ,"都電雑司ケ谷駅"  ,"鶯谷駅"     
    ,"上中里駅"       ,"京成上野駅"     ,"西ケ原駅"       ,"高田馬場駅" 
    ,"快速上野駅"     ,"田園調布駅"     ,"奥沢駅"         ,"狛江駅"     
    ,"緑が丘駅"       ,"大岡山駅"       ,"雪が谷大塚駅"    ,"牛込柳町駅" 
    ,"西巣鴨駅"       ,"面影橋駅"       ,"仲御徒町駅"     ,"浅草橋駅"   
    ,"蔵前駅"         ,"新御徒町駅"     ,"三ノ輪駅"       ,"入谷駅"     
    ,"馬喰町駅"       ,"南千住駅"       ,"三ノ輪橋駅"     ,"淡路町駅"   
    ,"小伝馬町駅"     ,"稲荷町駅"       ,"快速三河島駅"    ,"浅草駅"     
    ,"田原町駅"       ,"快速南千住駅"    ,"東日本橋駅"     ,"岩本町駅"   
    ,"両国駅"         ,"曳舟駅"         ,"本所吾妻橋駅"    ,"とうきょうスカイツリー駅"    
    ,"馬喰横山駅"     ,"東向島駅"        ,"押上駅"        ,"千住大橋駅" 
    ,"荒川一中前駅"    ,"北千住駅"       ,"人形町駅"       ,"浜町駅"     
    ,"東北沢駅"       ,"代々木上原駅"    ,"二子新地駅"     ,"目黒駅"      
    ,"五反田駅"       ,"恵比寿駅"       ,"不動前駅"       ,"白金台駅" 
    ,"洗足駅"         ,"北千束駅"       ,"西小山駅"       ,"駒場東大前駅"
    ,"中目黒駅"       ,"都立大学駅"     ,"武蔵小山駅"     ,"代官山駅"   
    ,"渋谷駅"         ,"神泉駅"         ,"つつじケ丘駅"    ,"多摩川駅"   
    ,"石川台駅"       ,"高津駅"         ,"三鷹台駅"       ,"富士見ケ丘駅"
    ,"幡ケ谷駅"       ,"西永福駅"       ,"高井戸駅"       ,"永福町駅"   
    ,"新宿駅"         ,"御嶽山駅"       ,"浜田山駅"       ,"和泉多摩川駅"
    ,"代々木駅"       ,"千駄ケ谷駅"     ,"北参道駅"       ,"南新宿駅"   
    ,"広尾駅"         ,"新宿三丁目駅"    ,"初台駅"         ,"西新宿五丁目駅"
    ,"原宿駅"         ,"明治神宮前駅"    ,"代々木公園駅"    ,"代々木八幡駅"
    ,"参宮橋駅"       ,"中野新橋駅"     ,"方南町駅"       ,"表参道駅"   
    ,"中野坂上駅"     ,"都庁前駅"       ,"外苑前駅"       ,"中野富士見町駅"
    ,"信濃町駅"       ,"白金高輪駅"     ,"国立競技場駅"    ,"西新宿駅"   
    ,"椎名町駅"       ,"要町駅"         ,"北池袋駅"       ,"板橋駅"     
    ,"東長崎駅"       ,"千川駅"         ,"小竹向原駅"     ,"下板橋駅"   
    ,"落合南長崎駅"    ,"巣鴨新田駅"     ,"大山駅"         ,"庚申塚駅"   
    ,"新板橋駅"       ,"新江古田駅"     ,"中井駅"         ,"新庚申塚駅" 
    ,"江古田駅"       ,"下落合駅"       ,"学習院下駅"     ,"西ケ原四丁目駅"
    ,"板橋区役所前駅"    ,"東中野駅"      ,"王子駅"         ,"蓮根駅"     
    ,"浮間舟渡駅"     ,"西台駅"         ,"下赤塚駅"       ,"地下鉄赤塚駅"
    ,"平和台駅"       ,"志村坂上駅"     ,"本蓮沼駅"       ,"志村三丁目駅"
    ,"北赤羽駅"   ,"西早稲田駅"         ,"新井薬師前駅"    ,"滝野川一丁目駅"
    ,"新三河島駅"     ,"尾久駅"         ,"赤羽駅"         ,"赤羽岩淵駅" 
    ,"十条駅"         ,"板橋本町駅"     ,"志茂駅"         ,"東十条駅"   
    ,"王子神谷駅"     ,"梶原駅"         ,"栄町駅"         ,"赤土小学校前駅"
    ,"小台駅"         ,"宮ノ前駅"       ,"荒川遊園地前駅"    ,"飛鳥山駅"   
    ,"荒川車庫前駅"    ,"戸田公園駅"     ,"西新井駅"       ,"熊野前駅"   
    ,"新桜台駅"       ,"上板橋駅"       ,"東武練馬駅"     ,"氷川台駅"   
    ,"高島平駅"       ,"ときわ台駅"     ,"成増駅"         ,"地下鉄成増駅"
    ,"新高島平駅"     ,"中板橋駅"       ,"和光市駅"       ,"西高島平駅" 
    ,"光が丘駅"       ,"練馬春日町駅"    ,"豊島園駅"       ,"上石神井駅" 
    ,"西荻窪駅"       ,"武蔵関駅"       ,"練馬駅"         ,"桜台駅"     
    ,"練馬高野台駅"    ,"富士見台駅"     ,"石神井公園駅"    ,"荻窪駅"     
    ,"中村橋駅"       ,"鷺ノ宮駅"       ,"東伏見駅"       ,"野方駅"     
    ,"都立家政駅"     ,"大泉学園駅"      ,"井荻駅"         ,"保谷駅"     
    ,"西武柳沢駅"     ,"上井草駅"       ,"朝霞駅"         ,"下井草駅"   
    ,"沼袋駅"         ,"高円寺駅"       ,"三鷹駅"         ,"ひばりケ丘駅"   
    ,"中野駅"         ,"井の頭公園駅"    ,"阿佐ケ谷駅" 
])

In [6]:
analytical_data = data[analytical_columns]

In [7]:
analytical_data.isna().sum().sum()

0

In [8]:
table_name = "analytical_data_table"
analytical_data.to_sql(table_name, con=engine, if_exists="replace")

In [9]:
with open('../../gci_project_storage/analytical_data.pickle', mode='wb') as f:
     pickle.dump(analytical_data, f)